In [1]:
# setup
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import requests
import calendar
import geopandas as gpd
import os.path as os
import scipy.stats
import seaborn.palettes
import seaborn.utils
import sys
from census import Census
from us import states
import http.client, urllib.request, urllib.parse, urllib.error, base64
import config
import quickstart

root= r"C:/Users/Jennah/Desktop/Code/machine-learning-final"
inp= os.join(root, "data", "2_intermediate")
out= os.join(root, "data", "3_clean")

In [2]:
from pydrive.auth import GoogleAuth

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # Creates local webserver and auto handles authentication.

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=570719947456-53haph7mjc317oel1ujh0kmjbfd499jf.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [4]:
from pydrive.drive import GoogleDrive
drive = GoogleDrive(gauth)
fileList = drive.ListFile({'q': "'1kiYbPtSYavHQKzyMWtXaLhP-JM2toXRm' in parents and trashed=false"}).GetList()
for file in fileList:
  print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "To Share"):
      fileID = file['id']

Title: fire_dispatch_tracts.csv, ID: 1psGrpBmGmNHPwM4lsRyiDX9tX5cf_UsO
Title: fire_dispatch.csv, ID: 1ulJlwpaezzq_aYV6jZzRWpmXLVlpO8qu
Title: dev_permits.csv, ID: 1mH_X0bLa_a1r-hpzwz43HpC7fFcayufp
Title: all_vacate.csv, ID: 1WfnTEjOaDNEb3E2vurXS_XkxJ0J1isdN
Title: pluto_df.csv, ID: 1j2KR6Tpa4CTA0k1omVfE52UYlMBMz17o
Title: fire_vacate.csv, ID: 11LV7007TbrVlfilkSvslDewQRZ9m0F-A
Title: ll84_avgenergy.csv, ID: 1NOKtbGyL3_TBSMp_eqybVaQc67t33yuo
Title: pluto_ct.csv, ID: 1ZLZZS79fi4Z0RFTNMRO6GfkNwughRuej
Title: fire_tract_sum.csv, ID: 16PukYkHO7Fgtbx26vdfG2sdUfDGreUnq
Title: fire_vacate_bbl.csv, ID: 1caDnIMowquAoJuZi1tDlyEqAS4HhXiU0
Title: speculation_hpd.csv, ID: 1yf86QR6yv2Z9Q6WNINW87RASUsdHPDl6
Title: dob_2012_2022.csv, ID: 1vBwNTewjAQcDOLDWIh8ZYerUJRLYKqFs
Title: hpd_2012_2022_raw.csv, ID: 18bAcCa68UVpH_FpIlFqNxG6pxb-UFD9g
Title: hpd_per_BBL.csv, ID: 13-4GIdxui77MIVhk6QmfBUdmTTZS1U3v
Title: hpd_per_census_tract.csv, ID: 1wuKDto3UeOAoSZW2-ALnlFndKfSQYtrC
Title: vac_pluto.csv, ID: 1MOBs-Mjg

# Load Fire Dispatch Data

In [5]:
fires = drive.CreateFile({'id': '16PukYkHO7Fgtbx26vdfG2sdUfDGreUnq'})
fires.GetContentFile('fire_dispatch.csv')  

df= pd.read_csv('fire_dispatch.csv').drop("Unnamed: 0", axis = 1).rename({"BoroCT2010":"boroct2010"}, axis = 1)
df["boroct2010"]= df["boroct2010"].astype(str)
df.head()

,boroct2010,engines_assigned_quantity,ladders_assigned_quantity,dispatch_response_seconds_qy,num_fire_ev
0,1000201,3.218978,2.094891,34.591241,139
1,1000202,3.271817,2.160229,39.273247,699
2,1000600,3.083815,2.089595,38.008671,346
3,1000700,2.450000,1.787500,43.131250,163
4,1000800,3.222222,2.093567,39.330409,344


## Load PLUTO (Tract Average/Sum)

In [6]:
csv = drive.CreateFile({'id': '1ZLZZS79fi4Z0RFTNMRO6GfkNwughRuej'})
csv.GetContentFile('pluto.csv')  

pluto= pd.read_csv('pluto.csv')
pluto.head()

,ct2010,lotarea,bldgarea,numbldgs,numfloors,unitsres,unitstotal,assessland,assesstot,exempttot,yearbuilt,holc_AB,holc_CD,holc_D,bldgclass,landuse
0,1.00,7481.995708,54564.699571,1.171674,6.480687,48.630901,50.030043,4.251514e+05,4.228507e+06,2.936200e+06,1887.223176,0.484979,0.381974,0.381974,A4,1.0
1,2.00,2537.735795,2148.512784,1.213778,2.086832,1.924716,1.985795,1.230113e+04,5.077698e+04,2.521578e+03,1940.862926,0.000000,0.991477,0.358665,B2,1.0
2,2.01,9834.592593,34004.333333,1.111111,6.000000,38.629630,39.481481,4.172500e+05,2.380184e+06,1.576786e+06,1908.962963,0.000000,0.962963,0.962963,C4,2.0
3,2.02,47680.541667,131918.791667,2.000000,8.072917,143.416667,144.833333,1.553912e+06,7.543499e+06,4.247797e+06,1923.791667,0.000000,0.916667,0.916667,D4,[3. 4.]
4,3.00,11368.606061,21258.590909,1.045455,3.333333,18.757576,19.318182,1.095267e+05,6.323296e+05,1.716179e+05,1906.151515,0.000000,0.712121,0.651515,B2,[2. 4.]


In [13]:
pluto= pd.read_csv(os.join(inp, "pluto_ct.csv"))
pluto.head().columns

Index(['ct2010', 'lotarea', 'bldgarea', 'numbldgs', 'numfloors', 'unitsres',
       'unitstotal', 'assessland', 'assesstot', 'exempttot', 'yearbuilt',
       'holc_AB', 'holc_CD', 'holc_D', 'landuse_2.0', 'landuse_1.0',
       'landuse_3.0', 'landuse_4.0', 'bldgclass_R1', 'bldgclass_A9',
       'bldgclass_B3', 'bldgclass_A5', 'bldgclass_B2', 'bldgclass_C0',
       'bldgclass_C2', 'bldgclass_A1', 'bldgclass_A3', 'bldgclass_B1',
       'bldgclass_R4', 'bldgclass_R6', 'bldgclass_B9', 'bldgclass_C1',
       'bldgclass_R3', 'bldgclass_A7', 'bldgclass_S2', 'bldgclass_D1',
       'bldgclass_A0', 'bldgclass_D7', 'bldgclass_C3', 'bldgclass_D6',
       'bldgclass_RM', 'bldgclass_R2', 'bldgclass_S4', 'bldgclass_A2',
       'bldgclass_S1', 'bldgclass_S9', 'bldgclass_RR', 'bldgclass_C7',
       'bldgclass_C4', 'bldgclass_K4', 'bldgclass_S3', 'bldgclass_C5',
       'bldgclass_C6', 'bldgclass_D4', 'bldgclass_RD', 'bldgclass_D3',
       'bldgclass_A4', 'bldgclass_D8', 'bldgclass_S5', 'bldgclass_S0',
 

In [11]:
pluto["boroct2010"]= pluto["borough"] + pluto["ct2010"].astype(str).str.replace(".", "", regex = False).str.pad(width = 6, fillchar = "0")

KeyError: 'borough'

In [8]:
# left join from pluto to capture all census tracts
df_pl= pluto.merge(df, how = "outer", on = "boroct2010", indicator = True)
print(df_pl["_merge"].value_counts())
df_pl.drop("_merge", axis = 1, inplace = True)

right_only    1911
left_only     1177
both           133
Name: _merge, dtype: int64


In [9]:
# fill missing values
df_pl[["engines_assigned_quantity", "ladders_assigned_quantity", "dispatch_response_seconds_qy", "num_fire_ev"]]= \
    df_pl[["engines_assigned_quantity", "ladders_assigned_quantity", "dispatch_response_seconds_qy", "num_fire_ev"]].fillna(0)

In [58]:
df_pl["boroct_alt"]= "1" + df_pl["ct2010"].astype(str).str.replace(".00", "", regex = False).str.replace(".", "", regex = False).str.pad(width = 6, fillchar = "0")

## Load HPD Violations

In [59]:
csv = drive.CreateFile({'id': '1wuKDto3UeOAoSZW2-ALnlFndKfSQYtrC'})
csv.GetContentFile('hpd.csv')  

hpd= pd.read_csv('hpd.csv')
hpd.head()

,Borough,BoroID,CensusTract,HPD_Count
0,BRONX,2,2,1452
1,BRONX,2,4,2254
2,BRONX,2,16,1539
3,BRONX,2,19,3127
4,BRONX,2,20,3111


In [60]:
hpd= hpd.rename({"HPD_Count":"hpd_viol_count"}, axis = 1)
hpd["boroct_alt"]= hpd["BoroID"].astype(str) + hpd["CensusTract"].astype(str).str.pad(width = 6, fillchar = "0")
hpd["boroct_alt"]

0       2000002
1       2000004
2       2000016
3       2000019
4       2000020
         ...   
2097    5029104
2098    5030301
2099    5030302
2100    5031901
2101    5031902
Name: boroct_alt, Length: 2102, dtype: object

In [62]:
# left join from pluto to capture all census tracts
df_hpd= df_pl.merge(hpd, how = "left", on = "boroct_alt", indicator = True)
print(df_hpd["_merge"].value_counts())
df_hpd.drop("_merge", axis = 1, inplace = True)

left_only     3143
both            78
right_only       0
Name: _merge, dtype: int64


## Load PLUTO data and compute neighborhood averages

In [71]:
analysis_bbl= pd.read_csv(os.join(out, "analysis_bbl.csv")).drop("Unnamed: 0", axis = 1)
analysis_bbl["boroid"]= np.select( [analysis_bbl["borough"] == "QN",\
                               analysis_bbl["borough"] == "MN",\
                               analysis_bbl["borough"] == "BX",\
                               analysis_bbl["borough"] == "BK",\
                               analysis_bbl["borough"] == "SI"], ["4", "1", "2", "3", "5"])
analysis_bbl["boroct2010"]= analysis_bbl["boroid"] + analysis_bbl["ct2010"].astype(str).str.replace(".00", "", regex = False).str.replace(".", "", regex = False).str.pad(width = 6, fillchar = "0")

In [73]:
analysis_bbl.head()

,borough,cd,ct2010,zipcode,address,bldgclass,landuse,ownertype,ownername,lotarea,...,vacate_ind,num_vacate_orders,viol_count,viol_count_per_unit,AL,NB,average_energy_usage,average_energy_usage_mi,boroid,boroct2010
0,MN,102.0,61.0,10003.0,42 EAST 12 STREET,R1,2.0,NaN,UNAVAILABLE OWNER,2643.0,...,0,0.0,1.0,0.166667,0.0,0.0,0.0,1.0,1,1000610
1,BK,315.0,392.0,11223.0,2105 OCEAN PARKWAY,A9,1.0,NaN,OCEAN PARKWAY BH 26 LLC,3445.0,...,0,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,3,3003920
2,BK,315.0,388.0,11223.0,2287 EAST 1 STREET,B3,1.0,NaN,2287E1 LLC,1500.0,...,0,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,3,3003880
3,QN,408.0,450.0,11432.0,161-26 GRAND CENTRAL PKWY,A5,1.0,NaN,CARLOS RODRIGUEZ,2925.0,...,0,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,4,4004500
4,QN,413.0,1617.0,11426.0,244-14 91 AVENUE,A9,1.0,NaN,BASHIR IRFAN,6086.0,...,0,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,4,4016170


In [85]:
meancols= ["viol_count_per_unit", "AL", 'NB', "average_energy_usage", "average_energy_usage_mi"]
analysis_collapse= analysis_bbl.groupby("boroct2010")[meancols].mean().fillna(0).reset_index()
analysis_collapse.head()

,boroct2010,viol_count_per_unit,AL,NB,average_energy_usage,average_energy_usage_mi
0,1000060,0.000000,0.014109,0.001764,13.538823,0.880952
1,1000070,0.063843,0.013333,0.000000,73.762167,0.160000
2,1000080,0.000000,0.009804,0.000000,6.556985,0.952941
3,1000090,0.683030,0.032680,0.003268,24.934191,0.735294
4,1000120,0.948767,0.042735,0.000000,70.252564,0.384615


In [90]:
# left join
analysis_tract= df_hpd.merge(analysis_collapse, how = "left", on = "boroct2010", indicator = True)
print(analysis_tract["_merge"].value_counts())
analysis_tract.drop("_merge", axis = 1, inplace = True)

left_only     2544
both           677
right_only       0
Name: _merge, dtype: int64


In [95]:
analysis_tract= analysis_tract.drop(analysis_tract[analysis_tract["ct2010"].isna()].index, axis = 0)

In [96]:
analysis_tract.head()

,ct2010,lotarea,bldgarea,numbldgs,numfloors,unitsres,unitstotal,assessland,assesstot,exempttot,...,num_fire_ev,Borough,BoroID,CensusTract,hpd_viol_count,viol_count_per_unit,AL,NB,average_energy_usage,average_energy_usage_mi
0,1.00,7481.995708,54564.699571,1.171674,6.480687,48.630901,50.030043,4.251514e+05,4.228507e+06,2.936200e+06,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.00,2537.735795,2148.512784,1.213778,2.086832,1.924716,1.985795,1.230113e+04,5.077698e+04,2.521578e+03,...,0.0,MANHATTAN,1.0,20.0,2.0,NaN,NaN,NaN,NaN,NaN
2,2.01,9834.592593,34004.333333,1.111111,6.000000,38.629630,39.481481,4.172500e+05,2.380184e+06,1.576786e+06,...,139.0,MANHATTAN,1.0,201.0,963.0,2.939779,0.024691,0.0,0.000000,1.0
3,2.02,47680.541667,131918.791667,2.000000,8.072917,143.416667,144.833333,1.553912e+06,7.543499e+06,4.247797e+06,...,699.0,MANHATTAN,1.0,202.0,514.0,0.413023,0.018519,0.0,56.288021,0.5
4,3.00,11368.606061,21258.590909,1.045455,3.333333,18.757576,19.318182,1.095267e+05,6.323296e+05,1.716179e+05,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Export analytic file

In [97]:
analysis_tract.to_csv(os.join(root, "data", "3_clean", "analysis_tract.csv"))